# PROCESADO DEL CSV ORIGINAL DE KAGGLE PARA el TEST

In [1]:
import pandas as pd
import numpy as np

## Cargamos los datos de entrada del archivo .csv TEST

In [2]:
# Cargamos los datos de entrada del archivo TEST.csv
test = pd.read_csv('../data/Raw/test.csv', sep=',')
df_test=pd.DataFrame(test)
df_test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


- **PassengerId** : una identificación única para cada pasajero. Cada Id toma la forma gggg_pp donde gggg indica un grupo con el que viaja el pasajero y pp es su número dentro del grupo. Las personas en un grupo a menudo son miembros de la familia, pero no siempre.
- **HomePlanet** : el planeta del que partió el pasajero, normalmente su planeta de residencia permanente.
- **CryoSleep** :  indica si el pasajero eligió ponerse en animación suspendida durante la duración del viaje. Los pasajeros en criosueño están confinados en sus cabinas.
- **Cabin** : el número de cabina donde se hospeda el pasajero. Toma la forma cubierta/número/lado, donde lado puede ser P para babor o S para estribor.
- **Destination** : el planeta dónde desembarcará el pasajero.
- **Age** : la edad del pasajero.
- **VIP** : si el pasajero ha pagado por un servicio VIP especial durante el viaje.
- **RoomService, FoodCourt, ShoppingMall, Spa, VRDeck** monto que el pasajero ha facturado en cada uno de los muchos servicios de lujo del Spaceship df_train.
- **Name** : el nombre y apellido del pasajero.

In [3]:
#comprobamos los valors nulos
df_test.isnull().sum()

PassengerId       0
HomePlanet       87
CryoSleep        93
Cabin           100
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
Name             94
dtype: int64

In [4]:
#HomePlanet : en esta variable rellenamos aquellos datos missing con el planeta de origen del pasajero más común entre ellos, 
# para ello vemos que planetas hay y posteriormente calculamos la moda.
df_test['HomePlanet'].fillna(df_test['HomePlanet'].mode()[0], inplace = True)

- Cryosleep: para estos valores perdidos nos centramos en dos opciones : 
  
  - Si un pasajero tiene un gasto distinto de cero en cualquier categoría de gasto, supondremos que este pasajero no esta confinado en su cabina, por tanto rellenamos con un False.
  
  - Si un pasajero tiene un gasto de cero en todas las categorías de gastos, este pasajero estará en animación suspendida en su cabina, por tanto rellenamos con un True.

In [5]:
bill_colum = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']

mask = (df_test.CryoSleep.isna())&((df_test[bill_colum]>0).any(1))
df_test.loc[:,'CryoSleep'][mask] = False
df_test.loc[:,'CryoSleep'][df_test.loc[:,'CryoSleep'].isna()]= True

c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


- Missing Bills: completamos todas las facturas faltantes con 0. Tenemos en cuenta que aproximadamente la mitad de las personas a las que les falta alguna factura estan en modo crionizados, por lo que necesariamente tienen que tener en todas sus facturas un importe de 0.

In [6]:
df_test.loc[:,'RoomService'][df_test.loc[:,'RoomService'].isna()]= 0.0
df_test.loc[:,'FoodCourt'][df_test.loc[:,'FoodCourt'].isna()]= 0.0
df_test.loc[:,'ShoppingMall'][df_test.loc[:,'ShoppingMall'].isna()]= 0.0
df_test.loc[:,'Spa'][df_test.loc[:,'Spa'].isna()]= 0.0
df_test.loc[:,'VRDeck'][df_test.loc[:,'VRDeck'].isna()]= 0.0

c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/in

- Cabin : en esta variable rellenamos los missing con la cabina más popular entre los pasajeros. Los datos de cabina vienen dados de la forma cubierta/número/lado, donde lado puede ser P para babor o S para estribor, en mi caso me quedo solo con la informacion de si el pasajero se encuentra en babor o estribor, eliminando así la información adicional. Crearemos así una nueva columna con P o S, eliminando la columna Cabin.

In [7]:
df_test['Cabin'].fillna(df_test['Cabin'].mode()[0], inplace = True)
df_test['Cabin']= df_test['Cabin'].astype('string')

- VIP: la mayoría de los pasajeros no tienen contratados los servicios VIP, por lo que rellemos estos datos perdidos con un False.

In [8]:
df_test.loc[:,'VIP'][df_test.loc[:,'VIP'].isnull()]= False

c:\Users\Alberto\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


- Age: para esta variable numérica rellenamos aquellos valores missing con la edad mediana de los pasajeros.

In [9]:
# Calculamos la media y la mediana

edad_mediana = df_test['Age'].median()
edad_media = df_test['Age'].mean()

print("Edad Mediana:", edad_mediana)
print("Edad Mediana:", edad_media)

Edad Mediana: 26.0
Edad Mediana: 28.65814620162446


In [10]:
#Calculamos la mediana

df_test['Age'].fillna(df_test['Age'].median(), inplace = True)

In [11]:
# Hay 2 opciones en la columna 'HomePlanet'. Como son campos de tipo object, osea, string, los paso a 0 ó 1 con el metodo getdummies y los guardo en un dataframe temporal.
planetas=pd.get_dummies(df_test['HomePlanet'])
planetas

,Earth,Europa,Mars
0,1,0,0
1,1,0,0
2,0,1,0
3,0,1,0
4,1,0,0
...,...,...,...
4272,1,0,0
4273,1,0,0
4274,0,0,1
4275,0,1,0


In [12]:
df_test=pd.concat([df_test, planetas], axis=1)

In [13]:
df_test.isnull().sum()

PassengerId      0
HomePlanet       0
CryoSleep        0
Cabin            0
Destination     92
Age              0
VIP              0
RoomService      0
FoodCourt        0
ShoppingMall     0
Spa              0
VRDeck           0
Name            94
Earth            0
Europa           0
Mars             0
dtype: int64

In [14]:
# Separo la columna cabin en 3 para crear mas tarde 3 nuevas columnas con sus correspondientes valores sacados de esta
df_test['Cabin_Side'] = df_test['Cabin'].str.split("/").str[2]
df_test['Cabin_num'] = df_test['Cabin'].str.split("/").str[1]
df_test['Cabin_Deck'] = df_test['Cabin'].str.split("/").str[0]

In [15]:
# Separo la columna 'Cabin_Side' en 2, porq tiene 2 valores,P y S y las guardo 
cabinas = pd.get_dummies(df_test['Cabin_Side'], prefix='Cabin_Side')
df_test=pd.concat([df_test, cabinas], axis=1)

In [16]:
# Convertimos la columna Cabin_deck a numerica con getdummies y la guardo en columnas individuales separadas
cabinas = pd.get_dummies(df_test['Cabin_Deck'], prefix='Cabin_Deck')
df_test=pd.concat([df_test, cabinas], axis=1)

In [17]:
#Borramos las columnas que he ido trasformando y que ya no me sirven
df_test.drop('Cabin_Side', axis=1, inplace=True)
df_test.drop('HomePlanet', axis=1, inplace=True)
df_test.drop('Destination', axis=1, inplace=True)
df_test.drop('Name', axis=1, inplace=True)
df_test.drop('Cabin',axis=1, inplace=True)
df_test.drop('Cabin_Deck',axis=1, inplace=True)

#Transformamos la columna VIP, q esta cm string a tipo int
df_test['VIP'] = df_test['VIP'].replace({False: 0, True: 1}).astype(int)
df_test['Cabin_num']=df_test['Cabin_num'].astype(int)

#Transformamos la columna CryoSleep, q esta cm string a tipo int
df_test['CryoSleep'] = df_test['CryoSleep'].replace({False: 0, True: 1}).astype(int)
df_test['Cabin_num']=df_test['Cabin_num'].astype(int)

In [18]:
# Ahora si...ataframe limpio de valores nulos y sin tipos de campos String o Object oncompatibles a la hora de crear el modelo y entrenarlo
df_test.isnull().sum()

PassengerId     0
CryoSleep       0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Earth           0
Europa          0
Mars            0
Cabin_num       0
Cabin_Side_P    0
Cabin_Side_S    0
Cabin_Deck_A    0
Cabin_Deck_B    0
Cabin_Deck_C    0
Cabin_Deck_D    0
Cabin_Deck_E    0
Cabin_Deck_F    0
Cabin_Deck_G    0
Cabin_Deck_T    0
dtype: int64

## guardamos en un nuevo CSV todos los datos ya procesados para poder usarlos en los notebooks de los modelos

In [19]:
df_test.to_csv('../data/Processed/test_processed.csv', sep=',', index=False)